In [1]:
import xbow
from xbow import instances
import boto3
ec2_resource = boto3.resource('ec2', region_name='eu-west-1')

In [2]:
insts = ec2_resource.instances.filter(Filters=[{'Name': 'key-name', 'Values': ['MyDaskScheduler']}])
print list(insts)
for instance in insts:
    print instance.key_name, instance.launch_time

[ec2.Instance(id='i-00a027622c0f6e2cb')]
MyDaskScheduler 2018-02-11 17:23:04+00:00


In [3]:
inst = list(insts)[0]
ci = instances.ConnectedInstance(inst)

In [4]:
ci.exec_command('ls')
print(ci.output)

amber16
build_gromacs_2016.4.sh
build_gromacs.sh
cocojob.sh
dicker
install_cuda.sh
packages
popylar
scheduler.log
shared
TGTC
TGTC_GMX
tios



In [ ]:
ci.upload('penta_w.crd', 'shared/CoCo-MD_example2/penta_w.crd')

In [ ]:
ci.upload('penta_w.top', 'shared/CoCo-MD_example2/penta_w.top')
ci.upload('penta_w.pdb', 'shared/CoCo-MD_example2/penta_w.pdb')
ci.upload('min1.in', 'shared/CoCo-MD_example2/min1.in')
ci.upload('min2.in', 'shared/CoCo-MD_example2/min2.in')
ci.upload('md.in', 'shared/CoCo-MD_example2/md.in')

In [ ]:
ci.upload('dryrun_cocomd_2.py', 'shared/CoCo-MD_example2/dryrun_cocomd_2.py')

In [ ]:
ci.exec_command('sudo pip install xbowflow==0.0.4')
print(ci.output)

In [5]:
insts = ec2_resource.instances.filter(Filters=[{'Name': 'key-name', 'Values': ['MyDaskWorkers']}])
print list(insts)
for instance in insts:
    print instance.key_name, instance.launch_time

[ec2.Instance(id='i-0738c43cbe807a380'), ec2.Instance(id='i-0f19b95b56a087e95')]
MyDaskWorkers 2018-02-11 17:23:25+00:00
MyDaskWorkers 2018-02-11 17:23:25+00:00


In [6]:
from xbow import pools
sip = pools.SpotInstancePool('MyDaskWorkers')
print sip.status

ready


In [15]:
sip.exec_command('sudo pip install xbowflow==0.0.5')
print sip.outputs[1]

Downloading/unpacking xbowflow==0.0.5
  Running setup.py (path:/tmp/pip_build_root/xbowflow/setup.py) egg_info for package xbowflow
    * Checking Python version...
    * Python version OK!
    
  Found existing installation: xbowflow 0.0.4
    Uninstalling xbowflow:
      Successfully uninstalled xbowflow
  Running setup.py install for xbowflow
    * Checking Python version...
    * Python version OK!
    changing mode of build/scripts-2.7/xflow-run from 644 to 755
    changing mode of build/scripts-2.7/xflow-submit from 644 to 755
    
    changing mode of /usr/local/bin/xflow-run to 755
    changing mode of /usr/local/bin/xflow-submit to 755
Cleaning up...



In [13]:
sip.exec_command('python -c "import sys; print sys.path"')
print sip.outputs

["['', '/home/ubuntu/.local/lib/python2.7/site-packages/popylar-0.2-py2.7.egg', '/usr/lib/python2.7/dist-packages', '/usr/local/lib/python2.7/dist-packages/pymongo-3.5.1-py2.7-linux-x86_64.egg', '/usr/local/lib/python2.7/dist-packages/xbowflow-0.0.4-py2.7.egg', '/usr/lib/python2.7', '/usr/lib/python2.7/plat-x86_64-linux-gnu', '/usr/lib/python2.7/lib-tk', '/usr/lib/python2.7/lib-old', '/usr/lib/python2.7/lib-dynload', '/home/ubuntu/.local/lib/python2.7/site-packages', '/usr/local/lib/python2.7/dist-packages', '/usr/lib/python2.7/dist-packages/gtk-2.0']\n", "['', '/home/ubuntu/.local/lib/python2.7/site-packages/popylar-0.2-py2.7.egg', '/usr/lib/python2.7/dist-packages', '/usr/local/lib/python2.7/dist-packages/pymongo-3.5.1-py2.7-linux-x86_64.egg', '/usr/local/lib/python2.7/dist-packages/xbowflow-0.0.4-py2.7.egg', '/usr/lib/python2.7', '/usr/lib/python2.7/plat-x86_64-linux-gnu', '/usr/lib/python2.7/lib-tk', '/usr/lib/python2.7/lib-old', '/usr/lib/python2.7/lib-dynload', '/home/ubuntu/.loc

In [16]:
sip.terminate()